FRP

Raw Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load the dataset
df = pd.read_csv(r"E:\IPMA\FRP\FRP_2000_2024.csv")

# Ensure lowercase columns for consistency
df.columns = [col.lower() for col in df.columns]

# Convert acquisition date to datetime
df['acq_date'] = pd.to_datetime(df['acq_date'])

# === COUNTING POINTS AND UNIQUE DAYS ===
total_points = len(df)
unique_days = df['acq_date'].nunique()
print(f"Total FRP points in CSV: {total_points}")
print(f"Number of unique days: {unique_days}")

# Start the plot
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-12, 36, 34, 66])

# Add map features
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND, facecolor='seashell')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

# Plot all FRP points with a uniform color
ax.scatter(df['longitude'], df['latitude'], color='maroon', s=0.05, alpha=1, transform=ccrs.PlateCarree())

# Title only, no colorbar
plt.title("All FRP Points (2000–2024)")
plt.tight_layout()
plt.show()
df


In [ ]:
# Classification 4 levels  ≤50 MW (low intensity), 50 to ≤500 MW (moderate to high intensity), 500 ≤ 1000 MW (very high intensity),and >1000 MW (extreme intensity)

import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Patch

# Load and prepare data
df = pd.read_csv(r"E:\IPMA\FRP\FRP_2000_2024.csv")
df.columns = [col.lower() for col in df.columns]
df['acq_date'] = pd.to_datetime(df['acq_date'])

# Classification bins and labels
bins = [0, 50, 500, 1000, float('inf')]
labels = ['Low (≤50 MW)', 'Moderate (51–500 MW)', 'Very High (501–1000 MW)', 'Extreme (>1000 MW)']
colors = ['green', 'orange', 'red', 'purple']

# Classify FRP into intensity categories
df['intensity'] = pd.cut(df['frp'], bins=bins, labels=labels, include_lowest=True)

# Begin plot
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-12, 36, 34, 66])  # Study area extent

# Add map features
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND, facecolor='seashell')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

# Plot each category separately and create legend handles as colored squares
legend_handles = []
for label, color in zip(labels, colors):
    cat_df = df[df['intensity'] == label]
    scatter = ax.scatter(cat_df['longitude'], cat_df['latitude'], label=label, color=color,
                         s=0.05, alpha=1, transform=ccrs.PlateCarree())
    legend_handles.append(Patch(color=color, label=label))

# Add legend with custom handles (colored squares)
plt.legend(handles=legend_handles, title='FRP Intensity', loc='upper left')
plt.title("FRP Points Classified by Intensity (2000–2024)")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

# Load CSV
df = pd.read_csv(r"E:\IPMA\FRP\FRP_2000_2024.csv")
df.columns = [col.lower() for col in df.columns]
df['acq_date'] = pd.to_datetime(df['acq_date'])

# Filter FRP ≥ 50 MW
df = df[df['frp'] >= 50]

# Reset index for safety
df = df.reset_index(drop=True)

# Create unique point IDs for lat/lon combinations
df['point_id'] = df.groupby(['latitude', 'longitude']).ngroup()

# Create coordinates
time = pd.to_datetime(sorted(df['acq_date'].unique()))
point_ids = df['point_id'].unique()

# Prepare data array shape: (time, point)
time_index = {t: i for i, t in enumerate(time)}
point_index = {pid: i for i, pid in enumerate(point_ids)}

# Initialize data array with NaN
frp_data = np.full((len(time), len(point_ids)), np.nan)

# Fill data array
for _, row in df.iterrows():
    t_idx = time_index[row['acq_date']]
    p_idx = point_index[row['point_id']]
    frp_data[t_idx, p_idx] = row['frp']

# Coordinates for each point
point_df = df.drop_duplicates(subset=['point_id'])[['point_id', 'latitude', 'longitude']].set_index('point_id')
latitudes = point_df.loc[point_ids, 'latitude'].values
longitudes = point_df.loc[point_ids, 'longitude'].values

# Build xarray Dataset
ds = xr.Dataset(
    data_vars=dict(
        frp=(["time", "point"], frp_data)
    ),
    coords=dict(
        time=("time", time),
        point=("point", point_ids),
        latitude=("point", latitudes),
        longitude=("point", longitudes)
    ),
    attrs=dict(
        description="Fire Radiative Power (FRP) from 2000 to 2024, filtered at ≥ 50 MW",
        source="EUMETSAT/FRP CSV to NetCDF conversion",
        institution="Your Organization",
        creator="Your Name"
    )
)

# Save to NetCDF
output_path = r"E:\IPMA\FRP\FRP_2000_2024.nc"
ds.to_netcdf(output_path)
print(f"Saved to: {output_path}")


Plots

In [ ]:
# Time series for daily sum of FRP over defined regions

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from shapely.geometry import Point

# Load FRP CSV data
csv_path = r"E:\IPMA\FRP\FRP_2000_2024.csv"
df = pd.read_csv(csv_path)
df.columns = [col.lower() for col in df.columns]
df['acq_date'] = pd.to_datetime(df['acq_date'])

# Filter by Study Area and FRP threshold
lat_min, lat_max, lon_min, lon_max = 34, 66, -12, 36
df = df[(df['latitude'] >= lat_min) & (df['latitude'] <= lat_max) &
        (df['longitude'] >= lon_min) & (df['longitude'] <= lon_max) &
        (df['frp'] >= 50)]

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Define shapefile paths and region names
shapefile_paths = {
    "Spain": r"E:\IPMA\Countries\ESP\gadm41_ESP_0.shp",
    "Portugal": r"E:\IPMA\Countries\PRT\gadm41_PRT_0.shp",
    "Greece": r"E:\IPMA\Countries\GRC\gadm41_GRC_0.shp",
    "Italy": r"E:\IPMA\Countries\ITA\gadm41_ITA_0.shp",
}

# Region colors
region_colors = {
    "Study Area": "palevioletred",
    "Spain": "darkorange",
    "Portugal": "forestgreen",
    "Greece": "cornflowerblue",
    "Italy": "brown",
}

# Initialize result dictionary
region_series = {}

# 1. Study Area — no shapefile needed
daily_study = gdf.groupby('acq_date')['frp'].sum()
region_series["Study Area"] = daily_study

# 2. For each country using shapefiles
for region, shp_path in shapefile_paths.items():
    region_gdf = gpd.read_file(shp_path).to_crs("EPSG:4326")
    
    # Spatial join to find points within region
    region_points = gpd.sjoin(gdf, region_gdf, how="inner", predicate="intersects")
    
    # Sum FRP daily
    daily_region = region_points.groupby('acq_date')['frp'].sum()
    region_series[region] = daily_region

# Plotting
plot_order = ["Study Area", "Italy", "Spain", "Portugal", "Greece"]
plt.figure(figsize=(14, 7))

for name in plot_order:
    if name in region_series:
        series = region_series[name].sort_index()
        color = region_colors.get(name, "black")
        plt.plot(series.index, series, label=name, color=color)

plt.title("Daily Total FRP (MW) by Region")
plt.xlabel("Year")
plt.ylabel("FRP Total (MW)")
plt.legend()

# Set x-axis range
all_dates = pd.concat(region_series.values()).index
plt.xlim(all_dates.min(), all_dates.max())

# Format x-axis
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator(1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Time series for daily sum of FRP over defined regions separated plots

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import os

# Load the main FRP dataset
file_path = r"E:\IPMA\FRP\FRP_2000_2024.nc"
ds = xr.open_dataset(file_path)

# The variable of interest
frp_sum = ds["FRP_sum"]

# Define the Study Area by lat/lon slices (unchanged)
study_area_bounds = (34, 66, -12, 36)  # lat_min, lat_max, lon_min, lon_max
study_area_color = "palevioletred"

# Folder where masks are stored
masks_folder = r"E:\IPMA\Countries"

# Mask region names (adjust if needed)
mask_names = ["Spain", "Portugal", "Greece", "Italy"]

# Colors for plotting
region_colors = {
    "Study Area": study_area_color,
    "Spain": "darkorange",
    "Portugal": "forestgreen",
    "Greece": "cornflowerblue",
    "Italy": "brown",
}

# Prepare dictionary to hold time series
region_series = {}

# 1) Study Area by lat/lon slice
lat_min, lat_max, lon_min, lon_max = study_area_bounds
study_subset = frp_sum.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
study_daily_total = study_subset.sum(dim=["latitude", "longitude"], skipna=True)
region_series["Study Area"] = pd.Series(study_daily_total.values, index=pd.to_datetime(study_daily_total["time"].values))

# 2) Other regions by masks
for region in mask_names:
    mask_path = os.path.join(masks_folder, f"{region}_mask.nc")
    mask_ds = xr.open_dataset(mask_path)
    mask_var_name = list(mask_ds.data_vars)[0]
    mask = mask_ds[mask_var_name]
    mask_bool = mask.astype(bool)
    masked_data = frp_sum.where(mask_bool)
    daily_total = masked_data.sum(dim=["latitude", "longitude"], skipna=True)
    region_series[region] = pd.Series(daily_total.values, index=pd.to_datetime(daily_total["time"].values))

# Define the plot order
plot_order = ["Study Area", "Italy", "Spain", "Portugal", "Greece"]

# Create subplots
fig, axes = plt.subplots(len(plot_order), 1, figsize=(12, 3 * len(plot_order)), sharex=True)

for i, name in enumerate(plot_order):
    series = region_series[name]
    color = region_colors.get(name, "black")
    axes[i].plot(series.index, series, label=name, color=color)
    axes[i].set_ylabel("FRP Total (MW)")
    axes[i].legend()
    axes[i].set_title(f"Daily Total FRP (MW) - {name}")

    # Format x-axis yearly
    axes[i].xaxis.set_major_locator(mdates.YearLocator(1))
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    axes[i].tick_params(axis='x', rotation=45)

# Set overall title and layout
plt.suptitle("Daily Total FRP (MW) by Region", y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
#Boxplot variability by month for every year

import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# === Load the main FRP dataset ===
file_path = r"E:\IPMA\FRP\FRP_2000_2024.nc"
ds = xr.open_dataset(file_path)

# === Month names for plotting ===
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# === Define Study Area bounds ===
study_area_bounds = (34, 66, -12, 36)  # lat_min, lat_max, lon_min, lon_max

# Folder where masks are stored
masks_folder = r"E:\IPMA\Countries"

# List of mask regions (adjust as needed)
mask_names = ["Spain", "Portugal", "Greece", "Italy"]

# Define colors consistent with previous plots
region_colors = {
    "Study Area": "palevioletred",
    "Spain": "darkorange",
    "Portugal": "forestgreen",
    "Greece": "cornflowerblue",
    "Italy": "brown",
}

# === Function to get daily mean time series for a masked region ===
def get_masked_region_daily_mean(frp_data, mask_ds):
    mask_var_name = list(mask_ds.data_vars)[0]
    mask = mask_ds[mask_var_name].astype(bool)
    masked_data = frp_data.where(mask)
    # Average over spatial dims, ignoring nan (masked points)
    return masked_data.mean(dim=["latitude", "longitude"], skipna=True)

# === Prepare regions dict with time series dataframes ===
region_dfs = {}

# 1) Study Area by lat/lon slice
lat_min, lat_max, lon_min, lon_max = study_area_bounds
study_subset = ds['FRP_sum'].sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
study_daily_mean = study_subset.mean(dim=["latitude", "longitude"], skipna=True)
df_study = study_daily_mean.to_dataframe(name='FRP_sum').reset_index()
region_dfs["Study Area"] = df_study

# 2) Other regions by mask
for region in mask_names:
    mask_path = os.path.join(masks_folder, f"{region}_mask.nc")
    mask_ds = xr.open_dataset(mask_path)
    daily_mean = get_masked_region_daily_mean(ds['FRP_sum'], mask_ds)
    df_region = daily_mean.to_dataframe(name='FRP_sum').reset_index()
    region_dfs[region] = df_region

# === Plot boxplots for each region ===
for region_name, df in region_dfs.items():
    # Extract month
    df['Month'] = df['time'].dt.month
    df['Month Name'] = df['Month'].apply(lambda m: month_names[m - 1])
    df["Month Name"] = pd.Categorical(df["Month Name"], categories=month_names, ordered=True)

    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Month Name", y="FRP_sum", data=df,
                showfliers=False, whis=1.5,
                color=region_colors.get(region_name, "gray"))  # Set consistent color here
    plt.title(f"Monthly Variability of Daily Total FRP\n{region_name}")
    plt.xlabel("Month")
    plt.ylabel("FRP Total (MW)")
    plt.tight_layout()
    plt.show()


In [ ]:
#Annual Total FRP

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# Load the main FRP dataset
file_path = r"E:\IPMA\FRP\FRP_2000_2024.nc"
ds = xr.open_dataset(file_path)

frp_sum = ds["FRP_sum"]

# Study Area bounds and color
study_area_bounds = (34, 66, -12, 36)
study_area_color = "palevioletred"

# Masks folder and mask regions (adjust as needed)
masks_folder = r"E:\IPMA\Countries"
mask_names = ["Spain", "Portugal", "Greece", "Italy"]

# Helper function to get annual sum from masked data
def annual_sum_masked_region(frp_data, mask_ds):
    mask_var_name = list(mask_ds.data_vars)[0]
    mask = mask_ds[mask_var_name].astype(bool)
    masked_data = frp_data.where(mask)
    # sum spatially (lat, lon) for each time
    spatial_sum = masked_data.sum(dim=["latitude", "longitude"], skipna=True)
    # convert to pandas series with datetime index
    ts = spatial_sum.to_series()
    ts.index = pd.to_datetime(ts.index)
    # group by year and sum
    return ts.groupby(ts.index.year).sum()

# 1) Study Area annual sum
lat_min, lat_max, lon_min, lon_max = study_area_bounds
study_subset = frp_sum.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
study_spatial_sum = study_subset.sum(dim=["latitude", "longitude"], skipna=True)
study_ts = study_spatial_sum.to_series()
study_ts.index = pd.to_datetime(study_ts.index)
study_annual_sum = study_ts.groupby(study_ts.index.year).sum()

# Plot Study Area
plt.figure(figsize=(10, 5))
plt.bar(study_annual_sum.index.astype(str), study_annual_sum.values, color=study_area_color)
plt.title("Annual Total FRP (MW) - Study Area")
plt.xlabel("Year")
plt.ylabel("Total FRP (MW)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2) Other regions
region_colors = {
    "Spain": "darkorange",
    "Portugal": "forestgreen",
    "Greece": "cornflowerblue",
    "Italy": "brown",
}

for region in mask_names:
    mask_path = os.path.join(masks_folder, f"{region}_mask.nc")
    mask_ds = xr.open_dataset(mask_path)
    annual_sums = annual_sum_masked_region(frp_sum, mask_ds)

    plt.figure(figsize=(10, 5))
    plt.bar(annual_sums.index.astype(str), annual_sums.values, color=region_colors.get(region, "black"))
    plt.title(f"Annual Total FRP (MW) - {region}")
    plt.xlabel("Year")
    plt.ylabel("Total FRP (MW)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
#FIRE DAYS AND NO FIRE DAYS

import xarray as xr
import numpy as np
import pandas as pd
import os

# File path for FRP data
file_path = r"E:/IPMA/FRP/FRP_2000_2024.nc"

# Folder with masks (one per region)
masks_folder = r"E:/IPMA/Countries"

# Regions and corresponding mask filenames
regions = {
    #"Study Area": None,  # no mask, will use lat/lon slice
    "Spain": "Spain_mask.nc",
    "Portugal": "Portugal_mask.nc",
    "Greece": "Greece_mask.nc",
    "Italy": "Italy_mask.nc",
}

# Study area bounds (lat_min, lat_max, lon_min, lon_max)
study_area_bounds = (34, 66, -12, 36)

# Load FRP dataset
ds = xr.open_dataset(file_path)
frp = ds["FRP_sum"]

# Total days in dataset
total_days = frp.sizes['time']
print(f"Total days in dataset: {total_days}\n")

results = {}

for region_name, mask_file in regions.items():
    if region_name == "Study Area":
        # Use lat/lon slice for Study Area
        lat_min, lat_max, lon_min, lon_max = study_area_bounds
        sub_data = frp.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
    else:
        # Load mask dataset
        mask_path = os.path.join(masks_folder, mask_file)
        mask_ds = xr.open_dataset(mask_path)
        mask_var = list(mask_ds.data_vars)[0]
        mask = mask_ds[mask_var].astype(bool)
        
        # Apply mask to frp data
        sub_data = frp.where(mask)
    
    # Sum over lat/lon for each time step
    daily_sum = sub_data.sum(dim=["latitude", "longitude"], skipna=True)
    
    # Fire days = days with total FRP > 0
    fire_days = (daily_sum > 500).sum().item()
    no_fire_days = total_days - fire_days
    
    results[region_name] = {
        "Fire Days": fire_days,
        "No Fire Days": no_fire_days,
    }

df = pd.DataFrame.from_dict(results, orient="index")
df.index.name = "Region"
df.reset_index(inplace=True)

print(df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Colors for fire status
colors = {
    "Fire Days": "tab:red",
    "No Fire Days": "tab:blue"
}

# Melt DataFrame for plotting
df_melted = df.melt(id_vars="Region", value_vars=["Fire Days", "No Fire Days"],
                    var_name="Status", value_name="Day Count")

# Get unique regions and statuses
regions = df["Region"].unique()
statuses = ["Fire Days", "No Fire Days"]
n_statuses = len(statuses)

# Bar settings
x = np.arange(len(regions))
bar_width = 0.35

# Create plot
fig, ax = plt.subplots(figsize=(10, 6))

for i, status in enumerate(statuses):
    subset = df_melted[df_melted["Status"] == status]
    values = subset["Day Count"].values
    ax.bar(x + i * bar_width, values, width=bar_width, label=status, color=colors.get(status, "gray"))

# Customization
ax.set_title("Number of Fire and No-Fire Days by Region", fontsize=14)
ax.set_ylabel("Number of Days")
ax.set_xlabel("Region")
ax.set_xticks(x + bar_width / 2)
ax.set_xticklabels(regions, rotation=45)
ax.legend(title="Status")

plt.tight_layout()
plt.show()


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# === INPUTS ===
frp_nc_path = r"E:\IPMA\FRP\FRP_2000_2024.nc"
masks_folder = r"E:\IPMA\Countries"

regions = {
    "Portugal": "Portugal_mask.nc",
    "Spain": "Spain_mask.nc",
    "Italy": "Italy_mask.nc",
    "Greece": "Greece_mask.nc",
}

# === Region colors ===
region_colors = {
    "Spain": "darkorange",
    "Portugal": "forestgreen",
    "Greece": "cornflowerblue",
    "Italy": "brown",
}

# === LOAD FRP DATA ===
ds = xr.open_dataset(frp_nc_path)
frp_count = ds["count"]  # [time, latitude, longitude]
time = pd.to_datetime(ds["time"].values)

# === PROCESS EACH REGION ===
for region, mask_filename in regions.items():
    print(f"Processing {region}...")

    # Load mask
    mask_path = os.path.join(masks_folder, mask_filename)
    mask_ds = xr.open_dataset(mask_path)
    mask_var = list(mask_ds.data_vars)[0]
    mask = mask_ds[mask_var].astype(bool)

    # Apply mask to FRP count
    masked_count = frp_count.where(mask)

    # Sum over lat/lon to get total daily hotspot count
    daily_sum = masked_count.sum(dim=["latitude", "longitude"], skipna=True)

    # Group by year and sum
    df = pd.DataFrame({
        "time": time,
        "hotspot_count": daily_sum.values
    })
    df["year"] = df["time"].dt.year
    yearly_counts = df.groupby("year")["hotspot_count"].sum()

    # === PLOT ===
    plt.figure(figsize=(10, 5))
    plt.bar(
        yearly_counts.index,
        yearly_counts.values,
        color=region_colors[region],
        alpha=0.85
    )
    plt.title(f"Yearly Number of Hotspots in {region}", fontsize=14)
    plt.xlabel("Year")
    plt.ylabel("Hotspot Count")
    plt.tight_layout()
    plt.show()


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os

# === INPUTS ===
frp_nc_path = r"E:\IPMA\FRP\FRP_2000_2024.nc"
masks_folder = r"E:\IPMA\Countries"

regions = {
    "Portugal": "Portugal_mask.nc",
    "Spain": "Spain_mask.nc",
    "Italy": "Italy_mask.nc",
    "Greece": "Greece_mask.nc",
}

# === Load FRP dataset ===
print("Loading FRP dataset...")
ds = xr.open_dataset(frp_nc_path)

# Calculate binary fire days: 1 if fire detected that day (FRP_sum > 0), else 0
fire_days_binary = (ds['FRP_sum'] > 500).astype(int)

# Sum over time to count fire days per pixel
print("Calculating fire days count per pixel...")
fire_days_count = fire_days_binary.sum(dim='time')

# === Loop through regions ===
for region_name, mask_file in regions.items():
    print(f"Processing region: {region_name}")

    # Load region mask
    mask_ds = xr.open_dataset(os.path.join(masks_folder, mask_file))
    mask = mask_ds['mask']

    # Align mask with fire_days_count grid (in case grids differ slightly)
    mask_aligned = mask.reindex_like(fire_days_count, method='nearest', tolerance=0.1)

    # Apply mask: keep values inside region, mask others as NaN
    fire_days_region = fire_days_count.where(mask_aligned == 1)

    # Plot
    plt.figure(figsize=(10, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    im = fire_days_region.plot(
        ax=ax,
        cmap='OrRd',
        cbar_kwargs={'label': f'Number of Fire Days in {region_name}'},
        vmin=0,
        vmax=fire_days_region.max().item()
    )
    ax.coastlines()
    ax.set_title(f'Fire Days Count per Pixel: {region_name}')
    plt.show()


Transform from point dataset to grid dataset

In [ ]:
# Turn point dataset into gridded dataset, where its horizontal resolution is 0.75º*0.75º

import pandas as pd
import numpy as np
import xarray as xr
import os

# === INPUT CSV FILE ===
csv_path = r"E:\IPMA\FRP\FRP_2000_2024.csv"

# === EXTRACT DIRECTORY AND CREATE OUTPUT FILENAME ===
base_dir = os.path.dirname(csv_path)
output_filename = "FRP_2000_2024_grid.nc"
output_path = os.path.join(base_dir, output_filename)

# === GRID PARAMETERS ===
lat_res = 0.75
lon_res = 0.75
lat_min, lat_max = 34, 66
lon_min, lon_max = -12, 36

lat_bins = np.arange(lat_min, lat_max + lat_res, lat_res)
lon_bins = np.arange(lon_min, lon_max + lon_res, lon_res)
lat_centers = lat_bins[:-1]
lon_centers = lon_bins[:-1]

# === LOAD AND PREPARE DATA ===
print("Loading and filtering data...")
df = pd.read_csv(csv_path)
df = df[(df['latitude'] >= lat_min) & (df['latitude'] <= lat_max) &
        (df['longitude'] >= lon_min) & (df['longitude'] <= lon_max)]

# Filter out low-intensity values
df = df[df['frp'] >= 50]

# Ensure date is datetime
df['acq_date'] = pd.to_datetime(df['acq_date'])

# Assign grid cell
print("Assigning points to grid cells...")
df['lat_bin'] = pd.cut(df['latitude'], bins=lat_bins, labels=lat_centers)
df['lon_bin'] = pd.cut(df['longitude'], bins=lon_bins, labels=lon_centers)
df.dropna(subset=['lat_bin', 'lon_bin'], inplace=True)

# Convert bins to float
df['lat_bin'] = df['lat_bin'].astype(float)
df['lon_bin'] = df['lon_bin'].astype(float)

# === AGGREGATE STATS ===
print("Aggregating FRP statistics...")
grouped = df.groupby(['acq_date', 'lat_bin', 'lon_bin'])

frp_stats = grouped['frp'].agg(
    FRP_sum='sum',
    FRP_mean='mean',
    FRP_median='median',
    FRP_max='max',
    FRP_min='min',
    FRP_std='std',
    count='count'
).reset_index()

# === CREATE FULL 3D ARRAYS ===
print("Creating 3D arrays...")

all_dates = pd.date_range(df['acq_date'].min(), df['acq_date'].max(), freq='D')
shape = (len(all_dates), len(lat_centers), len(lon_centers))

# Initialize arrays with NaNs
arrays = {
    'FRP_sum': np.full(shape, np.nan),
    'FRP_mean': np.full(shape, np.nan),
    'FRP_median': np.full(shape, np.nan),
    'FRP_max': np.full(shape, np.nan),
    'FRP_min': np.full(shape, np.nan),
    'FRP_std': np.full(shape, np.nan),
    'count': np.full(shape, np.nan),
}

lat_to_idx = {lat: i for i, lat in enumerate(lat_centers)}
lon_to_idx = {lon: i for i, lon in enumerate(lon_centers)}
date_to_idx = {date: i for i, date in enumerate(all_dates)}

# Fill arrays
for _, row in frp_stats.iterrows():
    d, lat, lon = row['acq_date'], row['lat_bin'], row['lon_bin']
    i, j, k = date_to_idx[d], lat_to_idx[lat], lon_to_idx[lon]
    for key in arrays:
        arrays[key][i, j, k] = row[key]

# Replace NaNs with 0 after filling arrays
for key in arrays:
    arrays[key] = np.nan_to_num(arrays[key], nan=0.0)


# === CREATE xarray.Dataset ===
print("Building xarray dataset...")
ds = xr.Dataset(
    {key: (["time", "latitude", "longitude"], arrays[key]) for key in arrays},
    coords={
        "time": all_dates,
        "latitude": lat_centers,
        "longitude": lon_centers,
    },
    attrs={
        "title": "Daily Gridded FRP Statistics (0.75° × 0.75°)",
        "source_file": os.path.basename(csv_path),
        "grid_resolution": "0.75 degrees",
        "description": "Includes daily FRP sum, mean, median, max, min, std, and detection count per grid cell"
    }
)

# === SAVE TO NETCDF ===
print(f"Saving to NetCDF: {output_path}")
ds.to_netcdf(output_path)
print("Done.")

ds

| Variable         | Description                                                                                                                                     |
| ---------------- | ----------------------------------------------------------------------------------------------------------------------------------------------- |
| **`FRP_sum`**    | The **total FRP** (sum of all detected fire pixels) in that grid cell on that day. Represents **total fire energy**.                            |
| **`FRP_mean`**   | The **average FRP** of all fire points within that grid cell on that day. Useful for typical fire intensity.                                    |
| **`FRP_median`** | The **median FRP** value for the fire points in the grid cell on that day. More robust to outliers than the mean.                               |
| **`FRP_max`**    | The **maximum FRP** among all fire points in that grid cell on that day. Indicates the **most intense fire**.                                   |
| **`FRP_min`**    | The **minimum FRP** detected in that grid cell on that day (≥50 by filter). Could suggest presence of small/low-intensity fires.                |
| **`FRP_std`**    | The **standard deviation** of FRP values in that grid cell on that day. Measures **variability** in fire intensity.                             |
| **`count`**      | The **number of fire detections** (i.e., how many fire points were detected) in that grid cell on that day. Indicates fire frequency or extent. |


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load the NetCDF dataset
ds = xr.open_dataset(r"E:\IPMA\FRP\FRP_2000_2024_grid.nc")

# Sum over time dimension to get total FRP per grid cell over all days
frp_sum = ds['FRP_sum'].sum(dim='time')

# Plotting
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)

# Plot the summed FRP data
frp_sum.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='OrRd',
    cbar_kwargs={
        'label': 'Total FRP (MW)',
        'orientation': 'horizontal',
        'shrink': 0.8,   # scale the size (optional)
        'pad': 0.05
    }
)

plt.title("Total Gridded FRP (2000–2024)")
plt.tight_layout()
plt.show()
